## Basic Classical Generative Adversarial Networks 
Reference: https://realpython.com/generative-adversarial-networks/#discriminative-vs-generative-models

<img src="../../assets/Classical%20GAN.png" width="auto" height="500" alt="Alt text">

In [4]:
import torch
print(torch.__version__)

2.3.1+cu121
